In [5]:
pip install -r requirements.txt

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [83]:
# Get Product ids from coinbase api
%run -i 'gather_tickers.py'

1INCH: 1INCH-USD, 1INCH-GBP, 1INCH-EUR, 1INCH-BTC, 
AAVE: AAVE-USD, AAVE-GBP, AAVE-BTC, AAVE-EUR, 
ZRX: ZRX-EUR, ZRX-USD, ZRX-BTC, 
REP: REP-USD, REP-BTC, 
BTC: BTC-EUR, BTC-USD, BTC-GBP, BTC-USDC, BTC-USDT, 
CTSI: CTSI-USD, CTSI-BTC, 
LTC: LTC-GBP, LTC-BTC, LTC-USD, LTC-EUR, 
FORTH: FORTH-GBP, FORTH-USD, FORTH-EUR, FORTH-BTC, 
DAI: DAI-USDC, DAI-USD, 
FIL: FIL-BTC, FIL-EUR, FIL-USD, FIL-GBP, 
MIR: MIR-USD, MIR-BTC, MIR-GBP, MIR-EUR, 
NMR: NMR-EUR, NMR-BTC, NMR-USD, NMR-GBP, 
SKL: SKL-EUR, SKL-BTC, SKL-USD, SKL-GBP, 
SNX: SNX-BTC, SNX-EUR, SNX-USD, SNX-GBP, 
NU: NU-BTC, NU-USD, NU-EUR, NU-GBP, 
ICP: ICP-USD, ICP-USDT, ICP-GBP, ICP-BTC, ICP-EUR, 
XLM: XLM-USD, XLM-EUR, XLM-BTC, 
UNI: UNI-USD, UNI-BTC, 
ATOM: ATOM-USD, ATOM-BTC, 
NKN: NKN-BTC, NKN-USD, 
MANA: MANA-EUR, MANA-USD, MANA-BTC, MANA-ETH, MANA-USDC, 
UMA: UMA-EUR, UMA-USD, UMA-GBP, UMA-BTC, 
USDT: USDT-GBP, USDT-USDC, USDT-USD, USDT-EUR, 
REN: REN-BTC, REN-USD, 
XTZ: XTZ-GBP, XTZ-BTC, XTZ-EUR, XTZ-USD, 
BAND: BAND-BTC, BAND-EUR

In [1]:
## Run technical analysis and pattern analysis on historical data
## Requires talib installation to run
## Adding a few basic indicators for the sake of test but can easily add more

import time 
import numpy as np
import pandas as pd
import cbpro
import talib
import datetime
import utils.validator as v

public_client = cbpro.PublicClient()
acceptedGrans = {"1m":60,"5m":300,"15m":900,"1h":3600,"6h":21600,"1d":86400}


# For the sake of simplicity we will fail over if the requested data set is too big.
# For an enhanced product, including if we  
ids = input("Please enter product id(s), comma separated i.e.'ETH-BTC,LINK-ETH': ")
start = input("Start date in yyyy-mm-dd format: ")
end = input("End date in yyyy-mm-dd format: ")
granularity = input("Enter timeframe: [1m,5m,15m,1h,6h,1d]: ")

#Hardcoded
# ids="ETH-BTC,LINK-ETH"
# start ="2021-01-01"
# end = "2021-06-01"
# granularity= "1d"


gran = acceptedGrans[str(granularity)]

product_ids_to_add = ids.split(",")

# Creating a dictionary for future comparisons of data streams
ohlcv_dictonary = {}
for product_id in product_ids_to_add:
    ohlcv = public_client.get_product_historic_rates(product_id,start,end,gran)
    df = pd.DataFrame(ohlcv, columns = ['time', 'low','high','open','close','volume']) 

    ohlcv_dictonary[str(product_id)]= df

    #DOJI Technical pattern
    CDL3BLACKCROWS = talib.CDL3BLACKCROWS(df['open'], df['high'], df['low'], df['close'])
    MFI = talib.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=14)
    doji = talib.CDLDOJI(df['open'], df['high'], df['low'], df['close'])
    ohlcv_dictonary[str(product_id)]["doji_count"] = doji[doji!=0].size
    print(product_id + " has " + str(doji[doji!=0].size) + " doji candles from " + start + " to " + end + " on the " + granularity + "."  )

    #Cycle indicator
    real = talib.HT_DCPHASE(df['close'])
    print(product_id + " has " + str(real[real>100].size) + " candles above 250 hilbert dominant phase from " + start + " to " + end + " on the " + granularity + "."  )

    #Volume indicators
    onbalancevolume = talib.OBV(df['close'], df['volume'])
    chaikinadoscillator = talib.ADOSC(df['high'], df['low'], df['close'], df['volume'], fastperiod=3, slowperiod=10)
    
    

    time.sleep(1)# Don't want to send too many requests at a time to the public coinbase api






Please enter product id(s), comma separated: ETH-BTC,LINK-ETH,LINK-BTC
Start date in yyyy-mm-dd format: 2021-05-01
End date in yyyy-mm-dd format: 2021-06-01
Enter timeframe: ,5m,15m,1h,6h,1d]: 6h
ETH-BTC has 20 doji candles from 2021-05-01 to 2021-06-01 on the 6h.
ETH-BTC has 17 days above 250 hilbert dominant phase from 2021-05-01 to 2021-06-01 on the 6h.
LINK-ETH has 20 doji candles from 2021-05-01 to 2021-06-01 on the 6h.
LINK-ETH has 45 days above 250 hilbert dominant phase from 2021-05-01 to 2021-06-01 on the 6h.
LINK-BTC has 20 doji candles from 2021-05-01 to 2021-06-01 on the 6h.
LINK-BTC has 30 days above 250 hilbert dominant phase from 2021-05-01 to 2021-06-01 on the 6h.


In [78]:
# Export chart data to csv.
import zipfile
import os
if not os.path.exists('output'):
    os.makedirs('output')

    
zipname= ids+"_from_"+start+"_to_"+end+".zip"
with zipfile.ZipFile("output/"+zipname, 'w') as csv_zip:
    for key in ohlcv_dictonary:
        dataframe_to_write = ohlcv_dictonary[str(key)]
        print(key)
        csv_zip.writestr(key+".csv", dataframe_to_write.to_csv())

print("Exported csvs to zip.")
    

ETH-BTC
Exported csvs to zip.
